# 03-03. 기존 코드 그대로 사용하면서 속도 개선하는 팁

## cudf.pandas

기존 `script.py` 소스 코드를 실행할 때 `cudf.pandas` 모듈 옵션을 사용하여 실행할 수 있습니다.

```bash
python -m cudf.pandas script.py
```

또는 자신이 가지고 있는 Pandas 사용 코드 맨 앞에 아래처럼 한 줄만 추가하여 사용할 수 있습니다.

```python
%load_ext cudf.pandas
import pandas as pd
```

### Q1. 이 코드만 자신의 개발 환경에서 맨 앞에 실행해 보자. 잘 실행 되는가?

GPU 사용이 가능한 상태인지, 최신 버전 NVIDIA 드라이버, CUDA 툴킷, RAPIDS가 에러 없이 잘 설치되어 있는지 다시 확인해 보아야 합니다.

### Q2. 이후 기존 코드 모두 잘 작동되는가?

기존에 잘 작동하던 코드인데 `cudf.pandas` 한 줄 추가했더니 에러가 발생할 수 있습니다.  
* 예: 최신 NumPy 버전에서 더 이상 사용되지 않는데 NumPy 배열의 `values` 속성에 접근하려 할 때 에러가 발생하기도 합니다.

### Q3. cudf.pandas 넣었을 때와 뺐을 때 시간을 비교해 보자. 빨라졌는가?

실행은 되는데 오히려 느려진 경우도 있습니다.
컴퓨터 개발 환경, 데이터 크기나 사용 코드에 따라 다양한 원인에 의해 CPU만 사용하는 Pandas가 빠른 경우도 있습니다. 
처리할 데이터 용량이 크면 메모리(RAM) 용량도 보통 데이터의 크기에 비례해 커야 합니다. 
그래서 CSV 보다 용량이 작게 최적화된 Parquet을 사용하면 메모리가 제한된 T4 GPU에 연결된 Colab이나 Kaggle, 고용량 VRAM을 가진 NVIDIA GPU가 설치된 PC나 서버에서도 [OTTO 데이터](https://www.kaggle.com/competitions/otto-recommender-system)를 실행해 볼 수 있습니다.

**Pandas (CPU) vs cuDF (GPU) 시간 비교 예시**

| 기능 | Pandas 실행시간(초) | cuDF 실행시간(초) |
| --- | --- | --- |
| Parquet 파일 읽기 및 처리 | 13.8 (7.05) | 0.73 (2.10) |
| 그룹화 및 집계 | 3.07 (1.01) | 0.03 (0.05) |
| 요일별 그룹화 및 집계 | 4.05 (3.31) | 0.34 (0.43) |

위 표의 괄호 안의 실행 시간만 비교해봐도 각각 3배/20배/7배 이상 빠른 성능을 보입니다. 
이 결과로 볼 때, 코드 수정 없이 한 줄을 추가하거나 Pandas만 cuDF로 바꾸도 데이터 프레임 관련 연산에서 상당한 성능 향상을 기대할 수 있습니다.

### HOW IT WORKS
cudf.pandas 실행 원리 (Execution Flow)

cudf.pandas는 기존의 Pandas 코드를 수정하지 않고도 GPU 가속을 누릴 수 있게 해주는 하이브리드 엔진입니다.

핵심 작동 원리는 **GPU로 처리가 가능한 부분은 자동으로 변환하여 처리하고, 처리할 수 없는 부분은 CPU가 처리**하는 것입니다.

1.모듈 로드 (Module Spoofing)
코드가 실행될 때 가장 먼저 일어나는 단계입니다.

* **매직 모듈 로드:** cudf.pandas를 활성화하면, 시스템은 일반적인 Pandas 대신 '매직 모듈(Magic Module)'을 로드합니다.
* **프록시 역할:** 이 모듈은 겉보기엔 Pandas와 동일하지만, 실제로는 사용자의 코드와 GPU/CPU 사이를 연결해 주는 프록시(Proxy, 대리인) 역할을 수행합니다.

2.실행 및 분기 (Execution Path)
사용자가 Pandas 함수(또는 Pandas를 사용하는 외부 라이브러리)를 호출하면, 프록시가 이를 받아 처리가능 여부에 따라 두 가지 경로 중 하나를 선택합니다.

### A. 고속 경로 (Fast Path: GPU)

1.  **함수 호출:** 사용자가 함수를 호출합니다.
2.  **cuDF 실행:** 호출된 함수가 cuDF에서 지원하는 기능이라면, 데이터를 GPU에서 즉시 처리합니다.
3.  **결과 반환:** 매우 빠른 속도로 결과를 반환합니다.

### B. 안전 경로 (Fallback Path: CPU)
cuDF가 아직 지원하지 않는 복잡한 기능을 사용할 때 발생하는 과정입니다. 에러를 발생시키지 않고 자동으로 처리합니다.

1.  **지원 불가 감지:** cuDF가 해당 연산을 처리할 수 없음을 판단합니다.
2.  **CPU로 복사 (Copy to CPU):** 데이터를 GPU 메모리에서 CPU 메모리로 이동시킵니다.
3.  **Pandas 실행 (Fallback to pandas):** 원래의 Pandas(CPU)를 이용해 연산을 수행합니다.
4.  **GPU로 복귀 (Copy to GPU):** 연산이 완료된 결과를 다시 GPU 메모리로 가져옵니다.


![Execution Flow](https://docs.rapids.ai/api/cudf/stable/_images/cudf-pandas-execution-flow.png)


